# Composing techniques: Digital Dynamical Decoupling and Zero Noise Extrapolation

Noise in quantum computers can arise from a variety of sources, and sometimes applying multiple error mitigation techniques can be more beneficial than applying a single technique alone. 

Here we apply a combination of Digital Dynamical Decoupling (DDD) and Zero Noise Extrapolation (ZNE) to a randomized benchmarking (RB) task.

In [DDD](../guide/ddd.md), the input quantum circuit is modified by adding in gate sequences at regular intervals designed to reduce interaction between (i.e., decouple) the qubits from their environment. 

In [ZNE](../guide/zne.md), the expectation value of the observable of interest is computed at different noise levels, and subsequently the ideal expectation value is inferred by extrapolating the measured results to the zero-noise
limit. 
More information on the DDD and ZNE techniques can be found in the corresponding sections of the user guide (linked
above).

## Setup

We begin by importing the relevant modules and libraries required for the rest of this tutorial.

In [ ]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits
from mitiq import MeasurementResult, Observable, PauliString

## Task

We will demonstrate using DDD + ZNE on RB circuits, which are generated using Mitiq's built-in benchmarking circuit generation function, `generate_rb_circuits()`. 
More information on the RB protocol is available in the [Randomized Benchmarking section](https://qiskit.org/ecosystem/experiments/manuals/verification/randomized_benchmarking.html) of the [Qiskit Experiments Manual](https://qiskit.org/ecosystem/experiments/manuals). 
In this example we use a two-qubit RB circuit with a Clifford depth (number of Clifford groups) of 10.

In [ ]:
# Perhaps better to use something with more structure, e.g. GHZ
circuit = generate_rb_circuits(2, 10)[0]

## Noise model and executor

The noise in this example is a combination of depolarizing and readout errors, the latter of which are modeled as bit flips immediately prior to measurement. We use an [executor function](../guide/executors.md) to run the quantum circuit with the noise model applied.

In [ ]:
def execute(circuit: cirq.Circuit, noise_level: float = 0.002, p0: float = 0.05) -> MeasurementResult:
    """
    TO-DO: change noise models between non-Markovian noise and Markovian noise.

    Execute a circuit with depolarizing noise of strength ``noise_level`` and readout errors ...
    """
    measurements = circuit[-1] # Last operation in circuit
    circuit =  circuit[:-1] # Everything before the mmt
    circuit = circuit.with_noise(cirq.depolarize(noise_level))

    # "Readout noise" (applied to all qubits just before mmt)
    circuit.append(cirq.bit_flip(p0).on_each(circuit.all_qubits()))
    circuit.append(measurements)

    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=10000)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

## Observable

In this example, the observable of interest is $ZI + IZ$.

In [ ]:
obs = Observable(PauliString("ZI"), PauliString("IZ"))

For the circuit defined above, the ideal (noiseless) expectation value of the $ZI + IZ$ observable is 2, but as we will see, the unmitigated (noisy) result is impacted by depolarizing and readout errors.

In [ ]:
from functools import partial

ideal_exec = partial(execute, noise_level=0, p0=0)

ideal = obs.expectation(circuit, ideal_exec)
print("Ideal value:", "{:.5f}".format(ideal.real))

noisy = obs.expectation(circuit, execute)
print("Unmitigated noisy value:", "{:.5f}".format(noisy.real))

Next we choose our gate sequences to be used in the digital dynamical decoupling routine (DDD). 
More information on choosing appropriate sequences can be found in the [DDD theory](../guide/ddd-5-theory.md#common-examples-of-ddd-sequences) section of the user guide.

In [ ]:
# Double check that the circuit actually gets updated with the control sequences

In [ ]:
from mitiq import ddd

rule = ddd.rules.xyxy

# Pretty sure I need something like batched_executor or serial executor. 
#
ddd_executor = ddd.mitigate_executor(execute, rule=rule)

ddd_result = obs.expectation(circuit, ddd_executor) # Currently not supported?
print("Mitigated value obtained with DDD:", "{:.5f}".format(ddd_result.real))

We can see that REM improves the results, but errors remain.
For comparison, we then apply ZNE without REM.

In [ ]:
from mitiq import zne

zne_executor = zne.mitigate_executor(execute, observable=obs, scale_noise=zne.scaling.folding.fold_global)
zne_result = zne_executor(circuit)
print("Mitigated value obtained with ZNE:", "{:.5f}".format(zne_result.real))

Finally, we apply a combination of REM and ZNE.
REM is applied first to minimize the impact of measurement errors on the extrapolated result in ZNE.

In [ ]:
combined_executor = zne.mitigate_executor(ddd_executor, observable=obs, scale_noise=zne.scaling.folding.fold_global)

combined_result = combined_executor(circuit)
print("Mitigated value obtained with REM + ZNE:", "{:.5f}".format(combined_result.real))

From this example we can see that each technique affords some improvement, and the combination of REM and ZNE is more effective in mitigating errors than either technique alone.